In [11]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import import_ipynb
from Control_group_density_threshold import control_m, control_m2

In [12]:
nielsen15 = pd.read_csv('../../Nielsen/aggregated_nielsen_2015.csv')
nielsen16 = pd.read_csv('../..//Nielsen/aggregated_nielsen_2016.csv')
nielsen15['year'] = 2015
nielsen16['year'] = 2016

In [13]:
nielsen = pd.concat((nielsen15, nielsen16))
nielsen = nielsen[~nielsen.is_walmart]

In [14]:
## Entriy/exit dates
fandom = pd.read_csv('../data_collection/plein_de_data/fandom_traitées.csv', parse_dates=['Opening_date', 'Closing_date'])[['State', 'County_name', 'County_fips', 'Opening_date', 'Closing_date']]

# We drop the state in which we do not trust our data (some mistakes stil remain)
fandom = fandom[~np.isin(fandom.State, ('CA', 'GA', 'KS', 'LA', 'TX'))]

# We concentrate our study on the movements (entries & exits) during the fiscal years 2015 and 2016
movements = fandom[((fandom.Opening_date >= '2015-01-31') & (fandom.Opening_date <= '2017-01-31')) | ((fandom.Closing_date >= '2015-01-31') & (fandom.Closing_date <= '2017-01-31'))]
#movements['year'] = movements.Opening_date.dt.year
#movements['month'] = movements.Opening_date.dt.month

## FIRST MODEL : one month

Regression model :
$$ Price_{i, t} = \alpha + \beta \cdot treat_i + \gamma \cdot post_t + \delta \cdot treat_i \cdot post_{t} + \varepsilon_{i, t}$$

In [15]:
year = 2016
month = 1

In [16]:
# We choose to focus on milk prices
milk = nielsen[nielsen.product_group_descr == 'MILK']


# The control group is composed by all states where nothing (no entry nor exit) happened.
control = milk[~np.isin(milk.guessed_store_county_fips, movements)].copy()
print(f"Size of the control group: {len(control.guessed_store_county_fips.unique())}.")


# The treatment group is composed by the states where one entry took place in august 2016 and where this entry is the only movement
count = movements.groupby('County_fips').count()
count = count[count.Opening_date + count.Closing_date == 1] # No more than one movement in the treatement group
treatment_movements = movements[(np.isin(movements.County_fips, count.index)) & (movements.Opening_date.dt.year == year) & (movements.Opening_date.dt.month == month)]

treatment = milk[np.isin(milk.guessed_store_county_fips, treatment_movements.County_fips )].copy()
print(f"Size of the treatment group: {len(treatment.guessed_store_county_fips.unique())}.")


# We create our dummies for the regression
control['treat'] = False
control['post'] = (control.purchase_month > month) & (control.purchase_year == year)
treatment['treat'] = True
treatment['post'] = (treatment.purchase_month > month) & (treatment.purchase_year == year)


# Final dataset for the regression :

df = pd.concat((control, treatment))[['upc_price', 'treat', 'post']]
df = df[df.upc_price != 0]

Size of the control group: 2279.
Size of the treatment group: 15.


In [17]:
reg = smf.ols(formula='np.log(upc_price) ~ treat * post', data=df)
results = reg.fit()

In [18]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      np.log(upc_price)   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     220.3
Date:                Thu, 27 Oct 2022   Prob (F-statistic):          7.49e-142
Time:                        13:02:55   Log-Likelihood:                 1814.8
No. Observations:               44561   AIC:                            -3622.
Df Residuals:                   44557   BIC:                            -3587.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                      1.0080      0.001    673.336      0.000       1.005       1.011
treat[T.True]                 -0.0396      0.017     -2.366      0.018      -0.072      -0.007
post[T.True]                  -0.0561      0.002    -25.288      0.000      -0.060      -0.052
treat[T.True]:post[T.True]    -0.0121      0.025     -0.489      0.625      -0.060       0.036
==============================================================================
Omnibus:                     4270.644   Durbin-Watson:                   0.957
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            22451.454
Skew:                          -0.319   Prob(JB):                         0.00
Kurtosis:                       6.418   Cond. No.                         28.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## SECOND MODEL : all months

Regression model :
$$ Price_{i, t} = \alpha + \beta \cdot treat_i + \delta \cdot treat_i \cdot post_{t} + \varepsilon_{i, t}$$

#### Entire control group

In [22]:
# We choose to focus on milk prices
milk = nielsen[nielsen.product_group_descr == 'MILK']


# The control group is composed by all states where nothing (no entry nor exit) happened.
control = milk[~np.isin(milk.guessed_store_county_fips, movements)].copy()
print(f"Size of the control group: {len(control.guessed_store_county_fips.unique())}.")


# The treatment group is composed by the states where one entry took place in 2016 and where this entry is the only movement
count = movements.groupby('County_fips').count()
count = count[count.Opening_date + count.Closing_date == 1] # No more than one movement in the treatement group
treatment_movements = movements[(np.isin(movements.County_fips, count.index))]

treatment = milk[np.isin(milk.guessed_store_county_fips, treatment_movements.County_fips )].copy()
treatment = treatment.merge(treatment_movements, left_on='guessed_store_county_fips', right_on='County_fips')
print(f"Size of the treatment group: {len(treatment.guessed_store_county_fips.unique())}.")


# We create our dummies for the regression
control['treat'] = False
control['interaction'] = False
treatment['treat'] = True
treatment['interaction'] = (treatment.purchase_month > treatment.Opening_date.dt.month) & (treatment.purchase_year >= treatment.Opening_date.dt.year)


# Final dataset for the regression :

df = pd.concat((control, treatment))[['upc_price', 'treat', 'interaction']]
df = df[df.upc_price != 0]

Size of the control group: 2279.
Size of the treatment group: 89.


In [23]:
reg = smf.ols(formula='np.log(upc_price) ~ treat + interaction', data=df)
results = reg.fit()

In [24]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      np.log(upc_price)   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     12.01
Date:                Thu, 27 Oct 2022   Prob (F-statistic):           6.08e-06
Time:                        13:04:06   Log-Likelihood:                 1977.3
No. Observations:               46318   AIC:                            -3949.
Df Residuals:                   46315   BIC:                            -3922.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.9824      0.001    890.825      0.000       0.980       0.985
treat[T.True]           0.0240      0.006      3.810      0.000       0.012       0.036
interaction[T.True]    -0.0502      0.011     -4.709      0.000      -0.071      -0.029
==============================================================================
Omnibus:                     4417.753   Durbin-Watson:                   0.925
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            21484.694
Skew:                          -0.348   Prob(JB):                         0.00
Kurtosis:                       6.263   Cond. No.                         10.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [25]:
np.exp(results.params[2])-1

-0.048959088263504036

#### Control group with county density btw fisrt and third quartiles of treatment group density

In [26]:
# We choose to focus on milk prices
milk = nielsen[nielsen.product_group_descr == 'MILK']


# The control group is composed by all states where nothing (no entry nor exit) happened.
control = milk[~np.isin(milk.guessed_store_county_fips, movements)].copy()
print(f"Size of the control group: {len(control.guessed_store_county_fips.unique())}.")


# The treatment group is composed by the states where one entry took place in 2016 and where this entry is the only movement
count = movements.groupby('County_fips').count()
count = count[count.Opening_date + count.Closing_date == 1] # No more than one movement in the treatement group
treatment_movements = movements[(np.isin(movements.County_fips, count.index))]

treatment = milk[np.isin(milk.guessed_store_county_fips, treatment_movements.County_fips )].copy()
treatment = treatment.merge(treatment_movements, left_on='guessed_store_county_fips', right_on='County_fips')
print(f"Size of the treatment group: {len(treatment.guessed_store_county_fips.unique())}.")


# We create our dummies for the regression
control['treat'] = False
control['interaction'] = False
treatment['treat'] = True
treatment['interaction'] = (treatment.purchase_month > treatment.Opening_date.dt.month) & (treatment.purchase_year >= treatment.Opening_date.dt.year)


# Final dataset for the regression :

df = pd.concat((control_m2, treatment))[['upc_price', 'treat', 'interaction']]
df = df[df.upc_price != 0]

Size of the control group: 2279.
Size of the treatment group: 89.


In [27]:
reg = smf.ols(formula='np.log(upc_price) ~ treat + interaction', data=df)
results = reg.fit()

In [28]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      np.log(upc_price)   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     34.47
Date:                Thu, 27 Oct 2022   Prob (F-statistic):           1.12e-15
Time:                        13:04:13   Log-Likelihood:                 6569.6
No. Observations:               25221   AIC:                        -1.313e+04
Df Residuals:                   25218   BIC:                        -1.311e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.9645      0.001    786.153      0.000       0.962       0.967
treat[T.True]           0.0419      0.005      8.164      0.000       0.032       0.052
interaction[T.True]    -0.0502      0.009     -5.855      0.000      -0.067      -0.033
==============================================================================
Omnibus:                     2871.679   Durbin-Watson:                   0.692
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             8892.586
Skew:                          -0.599   Prob(JB):                         0.00
Kurtosis:                       5.651   Cond. No.                         7.85
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [29]:
np.exp(results.params[2])-1

-0.048959088263501926

#### Control group with county density btw min and max of treatment group density

In [30]:
# We choose to focus on milk prices
milk = nielsen[nielsen.product_group_descr == 'MILK']


# The control group is composed by all states where nothing (no entry nor exit) happened.
control = milk[~np.isin(milk.guessed_store_county_fips, movements)].copy()
print(f"Size of the control group: {len(control.guessed_store_county_fips.unique())}.")


# The treatment group is composed by the states where one entry took place in 2016 and where this entry is the only movement
count = movements.groupby('County_fips').count()
count = count[count.Opening_date + count.Closing_date == 1] # No more than one movement in the treatement group
treatment_movements = movements[(np.isin(movements.County_fips, count.index))]

treatment = milk[np.isin(milk.guessed_store_county_fips, treatment_movements.County_fips )].copy()
treatment = treatment.merge(treatment_movements, left_on='guessed_store_county_fips', right_on='County_fips')
print(f"Size of the treatment group: {len(treatment.guessed_store_county_fips.unique())}.")


# We create our dummies for the regression
control['treat'] = False
control['interaction'] = False
treatment['treat'] = True
treatment['interaction'] = (treatment.purchase_month > treatment.Opening_date.dt.month) & (treatment.purchase_year >= treatment.Opening_date.dt.year)


# Final dataset for the regression :

df = pd.concat((control_m, treatment))[['upc_price', 'treat', 'interaction']]
df = df[df.upc_price != 0]

Size of the control group: 2279.
Size of the treatment group: 89.


In [31]:
reg = smf.ols(formula='np.log(upc_price) ~ treat + interaction', data=df)
results = reg.fit()

In [32]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      np.log(upc_price)   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     15.20
Date:                Thu, 27 Oct 2022   Prob (F-statistic):           2.51e-07
Time:                        13:04:58   Log-Likelihood:                 7123.3
No. Observations:               74189   AIC:                        -1.424e+04
Df Residuals:                   74186   BIC:                        -1.421e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.9779      0.001   1194.237      0.000       0.976       0.979
treat[T.True]           0.0285      0.006      4.816      0.000       0.017       0.040
interaction[T.True]    -0.0502      0.010     -4.967      0.000      -0.070      -0.030
==============================================================================
Omnibus:                     7914.123   Durbin-Watson:                   0.847
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            27362.635
Skew:                          -0.529   Prob(JB):                         0.00
Kurtosis:                       5.781   Cond. No.                         13.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [33]:
np.exp(results.params[2])-1

-0.04895908826350359

## THIRD MODEL : adding time fixed effects

Regression model :
$$ Price_{i, t} = \alpha + \beta \cdot treat_i + \delta \cdot treat_i \cdot post_{t} + \sum_{\tau=Janv15}^{Dec16}\gamma_{\tau} \cdot \mathbb{1}(t=\tau) + \varepsilon_{i, t}$$

#### Entire control group

In [13]:
# We choose to focus on milk prices
milk = nielsen[nielsen.product_group_descr == 'MILK']


# The control group is composed by all states where nothing (no entry nor exit) happened.
control = milk[~np.isin(milk.guessed_store_county_fips, movements)].copy()
print(f"Size of the control group: {len(control.guessed_store_county_fips.unique())}.")


# The treatment group is composed by the states where one entry took place in 2016 and where this entry is the only movement
count = movements.groupby('County_fips').count()
count = count[count.Opening_date + count.Closing_date == 1] # No more than one movement in the treatement group
treatment_movements = movements[(np.isin(movements.County_fips, count.index))]

treatment = milk[np.isin(milk.guessed_store_county_fips, treatment_movements.County_fips )].copy()
treatment = treatment.merge(treatment_movements, left_on='guessed_store_county_fips', right_on='County_fips')
print(f"Size of the treatment group: {len(treatment.guessed_store_county_fips.unique())}.")


# We create our dummies for the regression
control['treat'] = False
control['interaction'] = False
control['time_fixed_effects'] = list(zip(control.purchase_year, control.purchase_month))
treatment['treat'] = True
treatment['interaction'] = (treatment.purchase_month > treatment.Opening_date.dt.month) & (treatment.purchase_year >= treatment.Opening_date.dt.year)
treatment['time_fixed_effects'] = list(zip(treatment.purchase_year, treatment.purchase_month))


# Final dataset for the regression :

df = pd.concat((control, treatment))[['upc_price', 'treat', 'interaction', 'time_fixed_effects']]
df = df[df.upc_price != 0]

Size of the control group: 2279.
Size of the treatment group: 89.


In [14]:
reg = smf.ols(formula='np.log(upc_price) ~ treat + interaction + C(time_fixed_effects)', data=df)
results = reg.fit()

In [15]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      np.log(upc_price)   R-squared:                       0.039
Model:                            OLS   Adj. R-squared:                  0.038
Method:                 Least Squares   F-statistic:                     74.25
Date:                Thu, 27 Oct 2022   Prob (F-statistic):               0.00
Time:                        10:13:55   Log-Likelihood:                 2875.8
No. Observations:               46318   AIC:                            -5700.
Df Residuals:                   46292   BIC:                            -5472.
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               1.0689      0.005    207.880      0.000       1.059       1.079
treat[T.True]                           0.0186      0.006      3.014      0.003       0.007       0.031
interaction[T.True]                    -0.0339      0.011     -3.216      0.001      -0.055      -0.013
C(time_fixed_effects)[T.(2015, 2)]     -0.0281      0.007     -3.864      0.000      -0.042      -0.014
C(time_fixed_effects)[T.(2015, 3)]     -0.0396      0.007     -5.457      0.000      -0.054      -0.025
C(time_fixed_effects)[T.(2015, 4)]     -0.0671      0.007     -9.209      0.000      -0.081      -0.053
C(time_fixed_effects)[T.(2015, 5)]     -0.0671      0.007     -9.193      0.000      -0.081      -0.053
C(time_fixed_effects)[T.(2015, 6)]     -0.0754      0.007    -10.361      0.000      -0.090      -0.061
C(time_fixed_effects)[T.(2015, 7)]     -0.0697      0.007     -9.569      0.000      -0.084      -0.055
C(time_fixed_effects)[T.(2015, 8)]     -0.0680      0.007     -9.349      0.000      -0.082      -0.054
C(time_fixed_effects)[T.(2015, 9)]     -0.0748      0.007    -10.259      0.000      -0.089      -0.061
C(time_fixed_effects)[T.(2015, 10)]    -0.0843      0.007    -11.566      0.000      -0.099      -0.070
C(time_fixed_effects)[T.(2015, 11)]    -0.0740      0.007    -10.135      0.000      -0.088      -0.060
C(time_fixed_effects)[T.(2015, 12)]    -0.0548      0.007     -7.511      0.000      -0.069      -0.040
C(time_fixed_effects)[T.(2016, 1)]     -0.0915      0.007    -12.550      0.000      -0.106      -0.077
C(time_fixed_effects)[T.(2016, 2)]     -0.1108      0.007    -15.167      0.000      -0.125      -0.096
C(time_fixed_effects)[T.(2016, 3)]     -0.1246      0.007    -17.075      0.000      -0.139      -0.110
C(time_fixed_effects)[T.(2016, 4)]     -0.1320      0.007    -18.077      0.000      -0.146      -0.118
C(time_fixed_effects)[T.(2016, 5)]     -0.1324      0.007    -18.141      0.000      -0.147      -0.118
C(time_fixed_effects)[T.(2016, 6)]     -0.1449      0.007    -19.856      0.000      -0.159      -0.131
C(time_fixed_effects)[T.(2016, 7)]     -0.1495      0.007    -20.456      0.000      -0.164      -0.135
C(time_fixed_effects)[T.(2016, 8)]     -0.1437      0.007    -19.635      0.000      -0.158      -0.129
C(time_fixed_effects)[T.(2016, 9)]     -0.1435      0.007    -19.609      0.000      -0.158      -0.129
C(time_fixed_effects)[T.(2016, 10)]    -0.1107      0.007    -15.108      0.000      -0.125      -0.096
C(time_fixed_effects)[T.(2016, 11)]    -0.1154      0.007    -15.747      0.000      -0.130      -0.101
C(time_fixed_effects)[T.(2016, 12)]     0.0214      0.007      2.926      0.003       0.007       0.036
===================================================================

In [16]:
np.exp(results.params[2])-1

-0.03333556562495743

#### Control group with county density btw fisrt and third quartiles of treatment group density

In [36]:
# We choose to focus on milk prices
milk = nielsen[nielsen.product_group_descr == 'MILK']


# The control group is composed by all states where nothing (no entry nor exit) happened.
control = milk[~np.isin(milk.guessed_store_county_fips, movements)].copy()
print(f"Size of the control group: {len(control.guessed_store_county_fips.unique())}.")


# The treatment group is composed by the states where one entry took place in 2016 and where this entry is the only movement
count = movements.groupby('County_fips').count()
count = count[count.Opening_date + count.Closing_date == 1] # No more than one movement in the treatement group
treatment_movements = movements[(np.isin(movements.County_fips, count.index))]

treatment = milk[np.isin(milk.guessed_store_county_fips, treatment_movements.County_fips )].copy()
treatment = treatment.merge(treatment_movements, left_on='guessed_store_county_fips', right_on='County_fips')
print(f"Size of the treatment group: {len(treatment.guessed_store_county_fips.unique())}.")


# We create our dummies for the regression
control['treat'] = False
control['interaction'] = False
control_m2['time_fixed_effects'] = list(zip(control_m2.purchase_year, control_m2.purchase_month))
treatment['treat'] = True
treatment['interaction'] = (treatment.purchase_month > treatment.Opening_date.dt.month) & (treatment.purchase_year >= treatment.Opening_date.dt.year)
treatment['time_fixed_effects'] = list(zip(treatment.purchase_year, treatment.purchase_month))


# Final dataset for the regression :

df = pd.concat((control_m2, treatment))[['upc_price', 'treat', 'interaction', 'time_fixed_effects']]
df = df[df.upc_price != 0]

Size of the control group: 2279.
Size of the treatment group: 89.


C:\Users\inesn\AppData\Local\Temp\ipykernel_28128\1126528254.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_m2['time_fixed_effects'] = list(zip(control_m2.purchase_year, control_m2.purchase_month))


In [37]:
reg = smf.ols(formula='np.log(upc_price) ~ treat + interaction + C(time_fixed_effects)', data=df)
results = reg.fit()

In [38]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      np.log(upc_price)   R-squared:                       0.075
Model:                            OLS   Adj. R-squared:                  0.074
Method:                 Least Squares   F-statistic:                     81.35
Date:                Thu, 27 Oct 2022   Prob (F-statistic):               0.00
Time:                        13:10:40   Log-Likelihood:                 7514.1
No. Observations:               25221   AIC:                        -1.498e+04
Df Residuals:                   25195   BIC:                        -1.476e+04
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               1.0717      0.006    193.133      0.000       1.061       1.083
treat[T.True]                           0.0320      0.005      6.445      0.000       0.022       0.042
interaction[T.True]                    -0.0212      0.008     -2.527      0.012      -0.038      -0.005
C(time_fixed_effects)[T.(2015, 2)]     -0.0351      0.008     -4.487      0.000      -0.050      -0.020
C(time_fixed_effects)[T.(2015, 3)]     -0.0536      0.008     -6.855      0.000      -0.069      -0.038
C(time_fixed_effects)[T.(2015, 4)]     -0.0682      0.008     -8.703      0.000      -0.084      -0.053
C(time_fixed_effects)[T.(2015, 5)]     -0.0805      0.008    -10.275      0.000      -0.096      -0.065
C(time_fixed_effects)[T.(2015, 6)]     -0.0793      0.008    -10.111      0.000      -0.095      -0.064
C(time_fixed_effects)[T.(2015, 7)]     -0.0743      0.008     -9.479      0.000      -0.090      -0.059
C(time_fixed_effects)[T.(2015, 8)]     -0.0841      0.008    -10.755      0.000      -0.099      -0.069
C(time_fixed_effects)[T.(2015, 9)]     -0.0825      0.008    -10.526      0.000      -0.098      -0.067
C(time_fixed_effects)[T.(2015, 10)]    -0.0952      0.008    -12.140      0.000      -0.111      -0.080
C(time_fixed_effects)[T.(2015, 11)]    -0.0853      0.008    -10.856      0.000      -0.101      -0.070
C(time_fixed_effects)[T.(2015, 12)]    -0.0657      0.008     -8.391      0.000      -0.081      -0.050
C(time_fixed_effects)[T.(2016, 1)]     -0.1206      0.008    -15.421      0.000      -0.136      -0.105
C(time_fixed_effects)[T.(2016, 2)]     -0.1433      0.008    -18.300      0.000      -0.159      -0.128
C(time_fixed_effects)[T.(2016, 3)]     -0.1507      0.008    -19.224      0.000      -0.166      -0.135
C(time_fixed_effects)[T.(2016, 4)]     -0.1610      0.008    -20.530      0.000      -0.176      -0.146
C(time_fixed_effects)[T.(2016, 5)]     -0.1664      0.008    -21.234      0.000      -0.182      -0.151
C(time_fixed_effects)[T.(2016, 6)]     -0.1811      0.008    -23.100      0.000      -0.197      -0.166
C(time_fixed_effects)[T.(2016, 7)]     -0.1818      0.008    -23.169      0.000      -0.197      -0.166
C(time_fixed_effects)[T.(2016, 8)]     -0.1725      0.008    -21.979      0.000      -0.188      -0.157
C(time_fixed_effects)[T.(2016, 9)]     -0.1540      0.008    -19.598      0.000      -0.169      -0.139
C(time_fixed_effects)[T.(2016, 10)]    -0.1406      0.008    -17.909      0.000      -0.156      -0.125
C(time_fixed_effects)[T.(2016, 11)]    -0.1482      0.008    -18.858      0.000      -0.164      -0.133
C(time_fixed_effects)[T.(2016, 12)]    -0.0495      0.008     -6.305      0.000      -0.065      -0.034
===================================================================

In [35]:
np.exp(results.params[2])-1

-0.04895908826350359

#### Control group with county density btw min and max of treatment group density

In [39]:
# We choose to focus on milk prices
milk = nielsen[nielsen.product_group_descr == 'MILK']


# The control group is composed by all states where nothing (no entry nor exit) happened.
control = milk[~np.isin(milk.guessed_store_county_fips, movements)].copy()
print(f"Size of the control group: {len(control.guessed_store_county_fips.unique())}.")


# The treatment group is composed by the states where one entry took place in 2016 and where this entry is the only movement
count = movements.groupby('County_fips').count()
count = count[count.Opening_date + count.Closing_date == 1] # No more than one movement in the treatement group
treatment_movements = movements[(np.isin(movements.County_fips, count.index))]

treatment = milk[np.isin(milk.guessed_store_county_fips, treatment_movements.County_fips )].copy()
treatment = treatment.merge(treatment_movements, left_on='guessed_store_county_fips', right_on='County_fips')
print(f"Size of the treatment group: {len(treatment.guessed_store_county_fips.unique())}.")


# We create our dummies for the regression
control['treat'] = False
control['interaction'] = False
control_m['time_fixed_effects'] = list(zip(control_m.purchase_year, control_m.purchase_month))
treatment['treat'] = True
treatment['interaction'] = (treatment.purchase_month > treatment.Opening_date.dt.month) & (treatment.purchase_year >= treatment.Opening_date.dt.year)
treatment['time_fixed_effects'] = list(zip(treatment.purchase_year, treatment.purchase_month))


# Final dataset for the regression :

df = pd.concat((control_m, treatment))[['upc_price', 'treat', 'interaction', 'time_fixed_effects']]
df = df[df.upc_price != 0]

Size of the control group: 2279.
Size of the treatment group: 89.


C:\Users\inesn\AppData\Local\Temp\ipykernel_28128\2964230536.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_m['time_fixed_effects'] = list(zip(control_m.purchase_year, control_m.purchase_month))


In [40]:
reg = smf.ols(formula='np.log(upc_price) ~ treat + interaction + C(time_fixed_effects)', data=df)
results = reg.fit()

In [41]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      np.log(upc_price)   R-squared:                       0.044
Model:                            OLS   Adj. R-squared:                  0.044
Method:                 Least Squares   F-statistic:                     136.1
Date:                Thu, 27 Oct 2022   Prob (F-statistic):               0.00
Time:                        13:11:45   Log-Likelihood:                 8772.1
No. Observations:               74189   AIC:                        -1.749e+04
Df Residuals:                   74163   BIC:                        -1.725e+04
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               1.0793      0.004    279.988      0.000       1.072       1.087
treat[T.True]                           0.0200      0.006      3.438      0.001       0.009       0.031
interaction[T.True]                    -0.0249      0.010     -2.503      0.012      -0.044      -0.005
C(time_fixed_effects)[T.(2015, 2)]     -0.0335      0.005     -6.142      0.000      -0.044      -0.023
C(time_fixed_effects)[T.(2015, 3)]     -0.0492      0.005     -9.044      0.000      -0.060      -0.039
C(time_fixed_effects)[T.(2015, 4)]     -0.0744      0.005    -13.639      0.000      -0.085      -0.064
C(time_fixed_effects)[T.(2015, 5)]     -0.0788      0.005    -14.422      0.000      -0.090      -0.068
C(time_fixed_effects)[T.(2015, 6)]     -0.0827      0.005    -15.158      0.000      -0.093      -0.072
C(time_fixed_effects)[T.(2015, 7)]     -0.0785      0.005    -14.357      0.000      -0.089      -0.068
C(time_fixed_effects)[T.(2015, 8)]     -0.0801      0.005    -14.693      0.000      -0.091      -0.069
C(time_fixed_effects)[T.(2015, 9)]     -0.0875      0.005    -16.010      0.000      -0.098      -0.077
C(time_fixed_effects)[T.(2015, 10)]    -0.0889      0.005    -16.277      0.000      -0.100      -0.078
C(time_fixed_effects)[T.(2015, 11)]    -0.0840      0.005    -15.340      0.000      -0.095      -0.073
C(time_fixed_effects)[T.(2015, 12)]    -0.0624      0.005    -11.438      0.000      -0.073      -0.052
C(time_fixed_effects)[T.(2016, 1)]     -0.1102      0.005    -20.228      0.000      -0.121      -0.100
C(time_fixed_effects)[T.(2016, 2)]     -0.1276      0.005    -23.379      0.000      -0.138      -0.117
C(time_fixed_effects)[T.(2016, 3)]     -0.1396      0.005    -25.568      0.000      -0.150      -0.129
C(time_fixed_effects)[T.(2016, 4)]     -0.1496      0.005    -27.393      0.000      -0.160      -0.139
C(time_fixed_effects)[T.(2016, 5)]     -0.1571      0.005    -28.771      0.000      -0.168      -0.146
C(time_fixed_effects)[T.(2016, 6)]     -0.1678      0.005    -30.739      0.000      -0.178      -0.157
C(time_fixed_effects)[T.(2016, 7)]     -0.1734      0.005    -31.760      0.000      -0.184      -0.163
C(time_fixed_effects)[T.(2016, 8)]     -0.1583      0.005    -28.974      0.000      -0.169      -0.148
C(time_fixed_effects)[T.(2016, 9)]     -0.1479      0.005    -27.037      0.000      -0.159      -0.137
C(time_fixed_effects)[T.(2016, 10)]    -0.1293      0.005    -23.637      0.000      -0.140      -0.119
C(time_fixed_effects)[T.(2016, 11)]    -0.1350      0.005    -24.681      0.000      -0.146      -0.124
C(time_fixed_effects)[T.(2016, 12)]    -0.0424      0.005     -7.765      0.000      -0.053      -0.032
===================================================================

In [42]:
np.exp(results.params[2])-1

-0.024562597288311894

## THIRD MODEL : adding entity effects

Regression model :
$$ Price_{i, t} = \alpha + \beta \cdot treat_i + \delta \cdot treat_i \cdot post_{t} + \sum_{\tau=Janv15}^{Dec16}\gamma_{\tau} \cdot \mathbb{1}(t=\tau) + \sum_{s \in USStates}\gamma_{s} \cdot \mathbb{1}(state = s) + \varepsilon_{i, t}$$

#### Entire control group

In [17]:
# We choose to focus on milk prices
milk = nielsen[nielsen.product_group_descr == 'MILK']


# The control group is composed by all states where nothing (no entry nor exit) happened.
control = milk[~np.isin(milk.guessed_store_county_fips, movements)].copy()
print(f"Size of the control group: {len(control.guessed_store_county_fips.unique())}.")


# The treatment group is composed by the states where one entry took place in 2016 and where this entry is the only movement
count = movements.groupby('County_fips').count()
count = count[count.Opening_date + count.Closing_date == 1] # No more than one movement in the treatement group
treatment_movements = movements[(np.isin(movements.County_fips, count.index))]

treatment = milk[np.isin(milk.guessed_store_county_fips, treatment_movements.County_fips )].copy()
treatment = treatment.merge(treatment_movements, left_on='guessed_store_county_fips', right_on='County_fips')
print(f"Size of the treatment group: {len(treatment.guessed_store_county_fips.unique())}.")


# We create our dummies for the regression
control['treat'] = False
control['interaction'] = False
control['time_fixed_effects'] = list(zip(control.purchase_month, control.purchase_year))
treatment['treat'] = True
treatment['interaction'] = (treatment.purchase_month > treatment.Opening_date.dt.month) & (treatment.purchase_year >= treatment.Opening_date.dt.year)
treatment['time_fixed_effects'] = list(zip(treatment.purchase_month, treatment.purchase_year))


# Final dataset for the regression :

df = pd.concat((control, treatment))[['upc_price', 'treat', 'interaction', 'time_fixed_effects', 'store_state']]
df = df[df.upc_price != 0]

Size of the control group: 2279.
Size of the treatment group: 89.


In [18]:
reg = smf.ols(formula='np.log(upc_price) ~ treat + interaction + C(time_fixed_effects) + C(store_state)', data=df)
results = reg.fit()

In [19]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      np.log(upc_price)   R-squared:                       0.171
Model:                            OLS   Adj. R-squared:                  0.170
Method:                 Least Squares   F-statistic:                     130.7
Date:                Thu, 27 Oct 2022   Prob (F-statistic):               0.00
Time:                        10:14:01   Log-Likelihood:                 6309.8
No. Observations:               46318   AIC:                        -1.247e+04
Df Residuals:                   46244   BIC:                        -1.182e+04
Df Model:                          73                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               1.0909      0.008    132.592      0.000       1.075       1.107
treat[T.True]                          -0.0057      0.006     -0.952      0.341      -0.017       0.006
interaction[T.True]                    -0.0206      0.010     -2.097      0.036      -0.040      -0.001
C(time_fixed_effects)[T.(1, 2016)]     -0.0917      0.007    -13.533      0.000      -0.105      -0.078
C(time_fixed_effects)[T.(2, 2015)]     -0.0284      0.007     -4.207      0.000      -0.042      -0.015
C(time_fixed_effects)[T.(2, 2016)]     -0.1110      0.007    -16.353      0.000      -0.124      -0.098
C(time_fixed_effects)[T.(3, 2015)]     -0.0400      0.007     -5.925      0.000      -0.053      -0.027
C(time_fixed_effects)[T.(3, 2016)]     -0.1259      0.007    -18.575      0.000      -0.139      -0.113
C(time_fixed_effects)[T.(4, 2015)]     -0.0666      0.007     -9.840      0.000      -0.080      -0.053
C(time_fixed_effects)[T.(4, 2016)]     -0.1318      0.007    -19.421      0.000      -0.145      -0.118
C(time_fixed_effects)[T.(5, 2015)]     -0.0672      0.007     -9.914      0.000      -0.081      -0.054
C(time_fixed_effects)[T.(5, 2016)]     -0.1336      0.007    -19.699      0.000      -0.147      -0.120
C(time_fixed_effects)[T.(6, 2015)]     -0.0755      0.007    -11.166      0.000      -0.089      -0.062
C(time_fixed_effects)[T.(6, 2016)]     -0.1461      0.007    -21.545      0.000      -0.159      -0.133
C(time_fixed_effects)[T.(7, 2015)]     -0.0694      0.007    -10.255      0.000      -0.083      -0.056
C(time_fixed_effects)[T.(7, 2016)]     -0.1507      0.007    -22.201      0.000      -0.164      -0.137
C(time_fixed_effects)[T.(8, 2015)]     -0.0674      0.007     -9.985      0.000      -0.081      -0.054
C(time_fixed_effects)[T.(8, 2016)]     -0.1449      0.007    -21.309      0.000      -0.158      -0.132
C(time_fixed_effects)[T.(9, 2015)]     -0.0745      0.007    -11.001      0.000      -0.088      -0.061
C(time_fixed_effects)[T.(9, 2016)]     -0.1436      0.007    -21.129      0.000      -0.157      -0.130
C(time_fixed_effects)[T.(10, 2015)]    -0.0842      0.007    -12.429      0.000      -0.097      -0.071
C(time_fixed_effects)[T.(10, 2016)]    -0.1119      0.007    -16.434      0.000      -0.125      -0.099
C(time_fixed_effects)[T.(11, 2015)]    -0.0745      0.007    -10.981      0.000      -0.088      -0.061
C(time_fixed_effects)[T.(11, 2016)]    -0.1165      0.007    -17.118      0.000      -0.130      -0.103
C(time_fixed_effects)[T.(12, 2015)]    -0.0546      0.007     -8.052      0.000      -0.068      -0.041
C(time_fixed_effects)[T.(12, 2016)]     0.0200      0.007      2.946      0.003       0.007       0.033
C(store_state)[T.AR]                    0.0264      0.010      2.63

#### Control group with county density btw fisrt and third quartiles of treatment group density

In [43]:
# We choose to focus on milk prices
milk = nielsen[nielsen.product_group_descr == 'MILK']


# The control group is composed by all states where nothing (no entry nor exit) happened.
control = milk[~np.isin(milk.guessed_store_county_fips, movements)].copy()
print(f"Size of the control group: {len(control.guessed_store_county_fips.unique())}.")


# The treatment group is composed by the states where one entry took place in 2016 and where this entry is the only movement
count = movements.groupby('County_fips').count()
count = count[count.Opening_date + count.Closing_date == 1] # No more than one movement in the treatement group
treatment_movements = movements[(np.isin(movements.County_fips, count.index))]

treatment = milk[np.isin(milk.guessed_store_county_fips, treatment_movements.County_fips )].copy()
treatment = treatment.merge(treatment_movements, left_on='guessed_store_county_fips', right_on='County_fips')
print(f"Size of the treatment group: {len(treatment.guessed_store_county_fips.unique())}.")


# We create our dummies for the regression
control['treat'] = False
control['interaction'] = False
control_m2['time_fixed_effects'] = list(zip(control_m2.purchase_month, control_m2.purchase_year))
treatment['treat'] = True
treatment['interaction'] = (treatment.purchase_month > treatment.Opening_date.dt.month) & (treatment.purchase_year >= treatment.Opening_date.dt.year)
treatment['time_fixed_effects'] = list(zip(treatment.purchase_month, treatment.purchase_year))


# Final dataset for the regression :

df = pd.concat((control_m2, treatment))[['upc_price', 'treat', 'interaction', 'time_fixed_effects', 'store_state']]
df = df[df.upc_price != 0]

Size of the control group: 2279.
Size of the treatment group: 89.


C:\Users\inesn\AppData\Local\Temp\ipykernel_28128\3235791165.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_m2['time_fixed_effects'] = list(zip(control_m2.purchase_month, control_m2.purchase_year))


In [44]:
reg = smf.ols(formula='np.log(upc_price) ~ treat + interaction + C(time_fixed_effects) + C(store_state)', data=df)
results = reg.fit()

In [45]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      np.log(upc_price)   R-squared:                       0.289
Model:                            OLS   Adj. R-squared:                  0.287
Method:                 Least Squares   F-statistic:                     144.3
Date:                Thu, 27 Oct 2022   Prob (F-statistic):               0.00
Time:                        13:14:27   Log-Likelihood:                 10844.
No. Observations:               25221   AIC:                        -2.154e+04
Df Residuals:                   25149   BIC:                        -2.096e+04
Df Model:                          71                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               1.1447      0.009    129.771      0.000       1.127       1.162
treat[T.True]                           0.0072      0.005      1.522      0.128      -0.002       0.017
interaction[T.True]                    -0.0110      0.007     -1.486      0.137      -0.026       0.004
C(time_fixed_effects)[T.(1, 2016)]     -0.1207      0.007    -17.592      0.000      -0.134      -0.107
C(time_fixed_effects)[T.(2, 2015)]     -0.0349      0.007     -5.085      0.000      -0.048      -0.021
C(time_fixed_effects)[T.(2, 2016)]     -0.1431      0.007    -20.837      0.000      -0.157      -0.130
C(time_fixed_effects)[T.(3, 2015)]     -0.0535      0.007     -7.804      0.000      -0.067      -0.040
C(time_fixed_effects)[T.(3, 2016)]     -0.1509      0.007    -21.948      0.000      -0.164      -0.137
C(time_fixed_effects)[T.(4, 2015)]     -0.0679      0.007     -9.882      0.000      -0.081      -0.054
C(time_fixed_effects)[T.(4, 2016)]     -0.1616      0.007    -23.493      0.000      -0.175      -0.148
C(time_fixed_effects)[T.(5, 2015)]     -0.0805      0.007    -11.715      0.000      -0.094      -0.067
C(time_fixed_effects)[T.(5, 2016)]     -0.1668      0.007    -24.268      0.000      -0.180      -0.153
C(time_fixed_effects)[T.(6, 2015)]     -0.0799      0.007    -11.613      0.000      -0.093      -0.066
C(time_fixed_effects)[T.(6, 2016)]     -0.1814      0.007    -26.378      0.000      -0.195      -0.168
C(time_fixed_effects)[T.(7, 2015)]     -0.0741      0.007    -10.779      0.000      -0.088      -0.061
C(time_fixed_effects)[T.(7, 2016)]     -0.1825      0.007    -26.516      0.000      -0.196      -0.169
C(time_fixed_effects)[T.(8, 2015)]     -0.0842      0.007    -12.274      0.000      -0.098      -0.071
C(time_fixed_effects)[T.(8, 2016)]     -0.1729      0.007    -25.109      0.000      -0.186      -0.159
C(time_fixed_effects)[T.(9, 2015)]     -0.0834      0.007    -12.127      0.000      -0.097      -0.070
C(time_fixed_effects)[T.(9, 2016)]     -0.1546      0.007    -22.426      0.000      -0.168      -0.141
C(time_fixed_effects)[T.(10, 2015)]    -0.0951      0.007    -13.816      0.000      -0.109      -0.082
C(time_fixed_effects)[T.(10, 2016)]    -0.1417      0.007    -20.573      0.000      -0.155      -0.128
C(time_fixed_effects)[T.(11, 2015)]    -0.0866      0.007    -12.559      0.000      -0.100      -0.073
C(time_fixed_effects)[T.(11, 2016)]    -0.1493      0.007    -21.664      0.000      -0.163      -0.136
C(time_fixed_effects)[T.(12, 2015)]    -0.0667      0.007     -9.710      0.000      -0.080      -0.053
C(time_fixed_effects)[T.(12, 2016)]    -0.0500      0.007     -7.255      0.000      -0.063      -0.036
C(store_state)[T.AR]                   -0.0317      0.013     -2.42

#### Control group with county density btw min and max of treatment group density

In [46]:
# We choose to focus on milk prices
milk = nielsen[nielsen.product_group_descr == 'MILK']


# The control group is composed by all states where nothing (no entry nor exit) happened.
control = milk[~np.isin(milk.guessed_store_county_fips, movements)].copy()
print(f"Size of the control group: {len(control.guessed_store_county_fips.unique())}.")


# The treatment group is composed by the states where one entry took place in 2016 and where this entry is the only movement
count = movements.groupby('County_fips').count()
count = count[count.Opening_date + count.Closing_date == 1] # No more than one movement in the treatement group
treatment_movements = movements[(np.isin(movements.County_fips, count.index))]

treatment = milk[np.isin(milk.guessed_store_county_fips, treatment_movements.County_fips )].copy()
treatment = treatment.merge(treatment_movements, left_on='guessed_store_county_fips', right_on='County_fips')
print(f"Size of the treatment group: {len(treatment.guessed_store_county_fips.unique())}.")


# We create our dummies for the regression
control['treat'] = False
control['interaction'] = False
control_m['time_fixed_effects'] = list(zip(control_m.purchase_month, control_m.purchase_year))
treatment['treat'] = True
treatment['interaction'] = (treatment.purchase_month > treatment.Opening_date.dt.month) & (treatment.purchase_year >= treatment.Opening_date.dt.year)
treatment['time_fixed_effects'] = list(zip(treatment.purchase_month, treatment.purchase_year))


# Final dataset for the regression :

df = pd.concat((control_m, treatment))[['upc_price', 'treat', 'interaction', 'time_fixed_effects', 'store_state']]
df = df[df.upc_price != 0]

Size of the control group: 2279.
Size of the treatment group: 89.


C:\Users\inesn\AppData\Local\Temp\ipykernel_28128\1463375533.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_m['time_fixed_effects'] = list(zip(control_m.purchase_month, control_m.purchase_year))


In [47]:
reg = smf.ols(formula='np.log(upc_price) ~ treat + interaction + C(time_fixed_effects) + C(store_state)', data=df)
results = reg.fit()

In [48]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      np.log(upc_price)   R-squared:                       0.179
Model:                            OLS   Adj. R-squared:                  0.178
Method:                 Least Squares   F-statistic:                     221.2
Date:                Thu, 27 Oct 2022   Prob (F-statistic):               0.00
Time:                        13:15:24   Log-Likelihood:                 14419.
No. Observations:               74189   AIC:                        -2.869e+04
Df Residuals:                   74115   BIC:                        -2.801e+04
Df Model:                          73                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               1.1253      0.006    187.640      0.000       1.114       1.137
treat[T.True]                          -0.0041      0.006     -0.750      0.453      -0.015       0.007
interaction[T.True]                    -0.0114      0.009     -1.238      0.216      -0.030       0.007
C(time_fixed_effects)[T.(1, 2016)]     -0.1101      0.005    -21.798      0.000      -0.120      -0.100
C(time_fixed_effects)[T.(2, 2015)]     -0.0336      0.005     -6.658      0.000      -0.044      -0.024
C(time_fixed_effects)[T.(2, 2016)]     -0.1271      0.005    -25.108      0.000      -0.137      -0.117
C(time_fixed_effects)[T.(3, 2015)]     -0.0492      0.005     -9.743      0.000      -0.059      -0.039
C(time_fixed_effects)[T.(3, 2016)]     -0.1396      0.005    -27.585      0.000      -0.150      -0.130
C(time_fixed_effects)[T.(4, 2015)]     -0.0737      0.005    -14.578      0.000      -0.084      -0.064
C(time_fixed_effects)[T.(4, 2016)]     -0.1491      0.005    -29.445      0.000      -0.159      -0.139
C(time_fixed_effects)[T.(5, 2015)]     -0.0786      0.005    -15.508      0.000      -0.089      -0.069
C(time_fixed_effects)[T.(5, 2016)]     -0.1574      0.005    -31.084      0.000      -0.167      -0.147
C(time_fixed_effects)[T.(6, 2015)]     -0.0829      0.005    -16.396      0.000      -0.093      -0.073
C(time_fixed_effects)[T.(6, 2016)]     -0.1680      0.005    -33.195      0.000      -0.178      -0.158
C(time_fixed_effects)[T.(7, 2015)]     -0.0781      0.005    -15.418      0.000      -0.088      -0.068
C(time_fixed_effects)[T.(7, 2016)]     -0.1740      0.005    -34.387      0.000      -0.184      -0.164
C(time_fixed_effects)[T.(8, 2015)]     -0.0797      0.005    -15.777      0.000      -0.090      -0.070
C(time_fixed_effects)[T.(8, 2016)]     -0.1581      0.005    -31.227      0.000      -0.168      -0.148
C(time_fixed_effects)[T.(9, 2015)]     -0.0875      0.005    -17.271      0.000      -0.097      -0.078
C(time_fixed_effects)[T.(9, 2016)]     -0.1473      0.005    -29.043      0.000      -0.157      -0.137
C(time_fixed_effects)[T.(10, 2015)]    -0.0886      0.005    -17.500      0.000      -0.099      -0.079
C(time_fixed_effects)[T.(10, 2016)]    -0.1293      0.005    -25.494      0.000      -0.139      -0.119
C(time_fixed_effects)[T.(11, 2015)]    -0.0843      0.005    -16.620      0.000      -0.094      -0.074
C(time_fixed_effects)[T.(11, 2016)]    -0.1351      0.005    -26.643      0.000      -0.145      -0.125
C(time_fixed_effects)[T.(12, 2015)]    -0.0623      0.005    -12.324      0.000      -0.072      -0.052
C(time_fixed_effects)[T.(12, 2016)]    -0.0423      0.005     -8.349      0.000      -0.052      -0.032
C(store_state)[T.AR]                    0.0114      0.007      1.62

## THIRD MODEL : adding entity effects

Regression model :
$$
\begin{align}
Price_{i, t} &= \alpha + \beta \cdot treat_i + \sum_{\tau=Feb15}^{Dec16} \delta_{\tau} \cdot treat_i \cdot \mathbb{1}(t=\tau)  + \sum_{\tau=Feb15}^{Dec16}\gamma_{\tau} \cdot \mathbb{1}(t=\tau) + \sum_{s \in USStates}\gamma_{s} \cdot \mathbb{1}(state = s) + \varepsilon_{i, t}\\
&= \beta_i + \sum_{\tau=Feb15}^{Dec16} \delta_{\tau} \cdot treat_i \cdot \mathbb{1}(t=\tau)  + \rho_t + \varepsilon_{i, t}
\end{align}
$$